In [59]:
import os
import re
import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pydicom as dicom
from sklearn.model_selection import train_test_split
import torch
import torchvision as tv
from sklearn.model_selection import GroupKFold
from torch.cuda.amp import GradScaler, autocast
from torchvision.models.feature_extraction import create_feature_extractor
from tqdm.notebook import tqdm
import timm

In [60]:
def load_dicom(path):
    """
    This supports loading both regular and compressed JPEG images.
    See the first sell with `pip install` commands for the necessary dependencies
    """
    img=dicom.dcmread(path)
    img.PhotometricInterpretation = 'YBR_FULL'
    data = img.pixel_array
    data = data - np.min(data)
    if np.max(data) != 0:
        data = data / np.max(data)
    data=(data * 255).astype(np.uint8)
    return cv2.cvtColor(data, cv2.COLOR_GRAY2RGB), img


In [61]:
RSNA_2022_PATH = '../input/rsna-2022-cervical-spine-fracture-detection'
TRAIN_IMAGES_PATH = f'{RSNA_2022_PATH}/train_images'
TEST_IMAGES_PATH = f'{RSNA_2022_PATH}/test_images'
EFFNET_MAX_TRAIN_BATCHES = 40000
EFFNET_MAX_EVAL_BATCHES = 200
ONE_CYCLE_MAX_LR = 0.0001
ONE_CYCLE_PCT_START = 0.3
SAVE_CHECKPOINT_EVERY_STEP = 1000
EFFNET_CHECKPOINTS_PATH = '../input/rsna-2022-base-effnetv2'
FRAC_LOSS_WEIGHT = 2.
N_FOLDS = 5
METADATA_PATH = '../input/vertebrae-detection-checkpoints'

PREDICT_MAX_BATCHES = 5000

DEVICE='cuda' if torch.cuda.is_available() else 'cpu'
if DEVICE == 'cuda':
    BATCH_SIZE = 32
else:
    BATCH_SIZE = 2

DEVICE

'cuda'

In [62]:

class EffnetDataSet(torch.utils.data.Dataset):
    def __init__(self, df, path, transforms=None):
        super().__init__()
        self.df = df
        self.path = path
        self.transforms = transforms

    def __getitem__(self, i):
        path = os.path.join(self.path, self.df.iloc[i].StudyInstanceUID, f'{self.df.iloc[i].Slice}.dcm')

        try:
            img = load_dicom(path)[0]
            # Pytorch uses (batch, channel, height, width) order. Converting (height, width, channel) -> (channel, height, width)
            # img = np.transpose(img, (2, 0, 1))
            if self.transforms is not None:
                img = self.transforms(image=img)['image']
            img = img.to(dtype=torch.float16)
        except Exception as ex:
            print(ex)
            return None

        if 'C1_fracture' in self.df:
            frac_targets = torch.as_tensor(self.df.iloc[i][['C1_fracture', 'C2_fracture', 'C3_fracture', 'C4_fracture',
                                                            'C5_fracture', 'C6_fracture', 'C7_fracture']].astype(
                'float32').values)
            vert_targets = torch.as_tensor(
                self.df.iloc[i][['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7']].astype('float32').values)
            frac_targets = frac_targets * vert_targets  # we only enable targets that are visible on the current slice
            return img, frac_targets, vert_targets
        return img

    def __len__(self):
        return len(self.df)

In [63]:
df_train = pd.read_csv(f'{RSNA_2022_PATH}/train.csv')
df_train_slices = pd.read_csv(r'W:\PycharmProjects\kaggle-RSNA\input\rsna-2022-spine-fracture-detection-metadata\train_segmented.csv')
c1c7 = [f'C{i}' for i in range(1, 8)]
df_train_slices[c1c7] = (df_train_slices[c1c7] > 0.5).astype(int)
df_train = df_train_slices.set_index('StudyInstanceUID').join(df_train.set_index('StudyInstanceUID'),
                                                              rsuffix='_fracture').reset_index().copy()
df_train = df_train.query('StudyInstanceUID != "1.2.826.0.1.3680043.20574"').reset_index(drop=True)

split = GroupKFold(N_FOLDS)
for k, (_, test_idx) in enumerate(split.split(df_train, groups=df_train.StudyInstanceUID)):
    df_train.loc[test_idx, 'split'] = k
df_train

,StudyInstanceUID,Slice,ImageHeight,ImageWidth,SliceThickness,ImagePositionPatient_x,ImagePositionPatient_y,ImagePositionPatient_z,C1,C2,...,C7,patient_overall,C1_fracture,C2_fracture,C3_fracture,C4_fracture,C5_fracture,C6_fracture,C7_fracture,split
0,1.2.826.0.1.3680043.10001,1,512,512,0.625,-52.308,-27.712,7.282,0,0,...,0,0,0,0,0,0,0,0,0,3.0
1,1.2.826.0.1.3680043.10001,2,512,512,0.625,-52.308,-27.712,6.657,0,0,...,0,0,0,0,0,0,0,0,0,3.0
2,1.2.826.0.1.3680043.10001,3,512,512,0.625,-52.308,-27.712,6.032,0,0,...,0,0,0,0,0,0,0,0,0,3.0
3,1.2.826.0.1.3680043.10001,4,512,512,0.625,-52.308,-27.712,5.407,0,0,...,0,0,0,0,0,0,0,0,0,3.0
4,1.2.826.0.1.3680043.10001,5,512,512,0.625,-52.308,-27.712,4.782,0,0,...,0,0,0,0,0,0,0,0,0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
711498,1.2.826.0.1.3680043.9997,251,512,512,0.625,-55.200,-24.600,-187.750,0,0,...,0,0,0,0,0,0,0,0,0,1.0
711499,1.2.826.0.1.3680043.9997,252,512,512,0.625,-55.200,-24.600,-188.375,0,0,...,0,0,0,0,0,0,0,0,0,1.0
711500,1.2.826.0.1.3680043.9997,253,512,512,0.625,-55.200,-24.600,-189.000,0,0,...,0,0,0,0,0,0,0,0,0,1.0
711501,1.2.826.0.1.3680043.9997,254,512,512,0.625,-55.200,-24.600,-189.625,0,0,...,0,0,0,0,0,0,0,0,0,1.0


In [64]:

def weighted_loss(y_pred_logit, y, reduction='mean', verbose=False):
    """
    Weighted loss
    We reuse torch.nn.functional.binary_cross_entropy_with_logits here. pos_weight and weights combined give us necessary coefficients described in https://www.kaggle.com/competitions/rsna-2022-cervical-spine-fracture-detection/discussion/340392

    See also this explanation: https://www.kaggle.com/code/samuelcortinhas/rsna-fracture-detection-in-depth-eda/notebook
    """

    neg_weights = (torch.tensor([7., 1, 1, 1, 1, 1, 1, 1]) if y_pred_logit.shape[-1] == 8 else torch.ones(y_pred_logit.shape[-1])).to(DEVICE)
    pos_weights = (torch.tensor([14., 2, 2, 2, 2, 2, 2, 2]) if y_pred_logit.shape[-1] == 8 else torch.ones(y_pred_logit.shape[-1]) * 2.).to(DEVICE)

    loss = torch.nn.functional.binary_cross_entropy_with_logits(
        y_pred_logit,
        y,
        reduction='none',
    )

    if verbose:
        print('loss', loss)

    pos_weights = y * pos_weights.unsqueeze(0)
    neg_weights = (1 - y) * neg_weights.unsqueeze(0)
    all_weights = pos_weights + neg_weights

    if verbose:
        print('all weights', all_weights)

    loss *= all_weights
    if verbose:
        print('weighted loss', loss)

    norm = torch.sum(all_weights, dim=1).unsqueeze(1)
    if verbose:
        print('normalization factors', norm)

    loss /= norm
    if verbose:
        print('normalized loss', loss)

    loss = torch.sum(loss, dim=1)
    if verbose:
        print('summed up over patient_overall-C1-C7 loss', loss)

    if reduction == 'mean':
        return torch.mean(loss)
    return loss


In [65]:
class EffnetModel(torch.nn.Module):
    def __init__(self):
        super().__init__()
        effnet = tv.models.efficientnet_v2_s()
        self.model = create_feature_extractor(effnet, ['flatten'])
        self.nn_fracture = torch.nn.Sequential(
            torch.nn.Linear(1280, 7),
        )
        self.nn_vertebrae = torch.nn.Sequential(
            torch.nn.Linear(1280, 7),
        )

    def forward(self, x):
        # returns logits
        x = self.model(x)['flatten']
        return self.nn_fracture(x), self.nn_vertebrae(x)

    def predict(self, x):
        frac, vert = self.forward(x)
        return torch.sigmoid(frac), torch.sigmoid(vert)


In [66]:
def filter_nones(b):
    return torch.utils.data.default_collate([v for v in b if v is not None])

def evaluate_effnet(model: EffnetModel, dl_test, max_batches=PREDICT_MAX_BATCHES, shuffle=False):
    torch.manual_seed(42)
    model = model.to(DEVICE)
    # dl_test = torch.utils.data.DataLoader(ds, batch_size=BATCH_SIZE, shuffle=shuffle, num_workers=os.cpu_count(),
                                          # collate_fn=filter_nones)
    pred_frac = []
    pred_vert = []
    with torch.no_grad():
        model.eval()
        frac_losses = []
        vert_losses = []
        with tqdm(dl_test, desc='Eval', miniters=10) as progress:
            for i, (X, y_frac, y_vert) in enumerate(progress):
                with autocast():
                    y_frac_pred, y_vert_pred = model.forward(X.to(DEVICE))
                    frac_loss = weighted_loss(y_frac_pred, y_frac.to(DEVICE)).item()
                    vert_loss = torch.nn.functional.binary_cross_entropy_with_logits(y_vert_pred, y_vert.to(DEVICE)).item()
                    pred_frac.append(torch.sigmoid(y_frac_pred))
                    pred_vert.append(torch.sigmoid(y_vert_pred))
                    frac_losses.append(frac_loss)
                    vert_losses.append(vert_loss)

                if i >= max_batches:
                    break
        return np.mean(frac_losses), np.mean(vert_losses), torch.concat(pred_frac).cpu().numpy(), torch.concat(pred_vert).cpu().numpy()

In [67]:
fold = 0
ds_train = EffnetDataSet(df_train.query('split != @fold'), TRAIN_IMAGES_PATH, A.Compose([
                      A.Resize(512, 512),
                      A.HorizontalFlip(p=0.5),
                      # # A.RandomContrast(p=0.5),
                      # # A.RandomBrightness(p=0.5),
                      # A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
                      # # A.RandomBrightness(limit=2, p=0.5),
                      # A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.05, rotate_limit=10, p=0.2),
                      #
                      # A.OneOf([
                      #     A.MotionBlur(p=0.2),
                      #     A.MedianBlur(blur_limit=3, p=0.1),
                      #     A.Blur(blur_limit=3, p=0.1),
                      # ], p=0.5),
                      ToTensorV2(),

                      ])
                  )

ds_val = EffnetDataSet(df_train.query('split == @fold'), TRAIN_IMAGES_PATH,A.Compose([
                      A.Resize(512, 512),
                      # A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
                      ToTensorV2(),
                      ])
                  )

# train_loader = torch.utils.data.DataLoader(ds_train, batch_size=16, shuffle=True, num_workers=10)
val_loader = torch.utils.data.DataLoader(ds_val, batch_size=1, shuffle=False, num_workers=0)

model = EffnetModel()

model.load_state_dict(torch.load('../output/ckpt-2d/ENV2_1012_512_loss0.1878.tph'))

frac_loss, vert_loss, effnet_pred_frac, effnet_pred_vert = evaluate_effnet(model, val_loader, PREDICT_MAX_BATCHES)

Eval:   0%|          | 0/142247 [00:00<?, ?it/s]

In [68]:
df_pred = []
fold = 0
df_effnet_pred = pd.DataFrame(data=np.concatenate([effnet_pred_frac, effnet_pred_vert], axis=1),
                              columns=[f'C{i}_effnet_frac' for i in range(1, 8)] +
                                      [f'C{i}_effnet_vert' for i in range(1, 8)])
df = pd.concat([df_train.query('split == @fold').head(len(df_effnet_pred)).reset_index(drop=True), df_effnet_pred],axis=1).sort_values(['StudyInstanceUID', 'Slice'])
df_pred.append(df)
df_pred = pd.concat(df_pred)
df_pred

,StudyInstanceUID,Slice,ImageHeight,ImageWidth,SliceThickness,ImagePositionPatient_x,ImagePositionPatient_y,ImagePositionPatient_z,C1,C2,...,C5_effnet_frac,C6_effnet_frac,C7_effnet_frac,C1_effnet_vert,C2_effnet_vert,C3_effnet_vert,C4_effnet_vert,C5_effnet_vert,C6_effnet_vert,C7_effnet_vert
0,1.2.826.0.1.3680043.10032,1,512,512,0.625,-82.000,11.712,37.166,0,0,...,0.000000,1.788139e-07,0.000008,1.788139e-07,1.490116e-06,5.960464e-08,0.000000,0.000000,0.000000,1.668930e-06
1,1.2.826.0.1.3680043.10032,2,512,512,0.625,-82.000,11.712,36.541,0,0,...,0.000000,5.960464e-08,0.000003,1.192093e-07,1.013279e-06,5.960464e-08,0.000000,0.000000,0.000000,2.384186e-07
2,1.2.826.0.1.3680043.10032,3,512,512,0.625,-82.000,11.712,35.916,0,0,...,0.000000,0.000000e+00,0.000002,5.960464e-08,7.748604e-07,5.960464e-08,0.000000,0.000000,0.000000,5.960464e-08
3,1.2.826.0.1.3680043.10032,4,512,512,0.625,-82.000,11.712,35.291,0,0,...,0.000000,0.000000e+00,0.000003,5.960464e-08,5.960464e-07,5.960464e-08,0.000000,0.000000,0.000000,1.192093e-07
4,1.2.826.0.1.3680043.10032,5,512,512,0.625,-82.000,11.712,34.666,0,0,...,0.000000,5.960464e-08,0.000004,5.960464e-08,6.556511e-07,5.960464e-08,0.000000,0.000000,0.000000,2.980232e-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4996,1.2.826.0.1.3680043.1102,258,512,512,0.625,-99.174,-64.166,-58.925,0,1,...,0.000052,8.230209e-04,0.000360,2.779007e-03,9.975586e-01,2.032518e-05,0.000009,0.000197,0.000125,6.061792e-05
4997,1.2.826.0.1.3680043.1102,259,512,512,0.625,-99.174,-64.166,-58.475,0,1,...,0.000057,8.726120e-04,0.000339,3.259659e-03,9.975586e-01,1.394749e-05,0.000008,0.000220,0.000131,5.918741e-05
4998,1.2.826.0.1.3680043.1102,260,512,512,0.625,-99.174,-64.166,-58.025,0,1,...,0.000048,9.002686e-04,0.000380,2.912521e-03,9.980469e-01,1.096725e-05,0.000005,0.000158,0.000127,7.724762e-05
4999,1.2.826.0.1.3680043.1102,261,512,512,0.625,-99.174,-64.166,-57.575,0,1,...,0.000036,9.002686e-04,0.000429,2.758026e-03,9.990234e-01,7.748604e-06,0.000003,0.000106,0.000116,9.840727e-05


In [69]:
target_cols = ['patient_overall'] + [f'C{i}_fracture' for i in range(1, 8)]
frac_cols = [f'C{i}_effnet_frac' for i in range(1, 8)]
vert_cols = [f'C{i}_effnet_vert' for i in range(1, 8)]


def patient_prediction(df):
    c1c7 = np.average(df[frac_cols].values, axis=0, weights=df[vert_cols].values)
    pred_patient_overall = 1 - np.prod(1 - c1c7)
    return np.concatenate([[pred_patient_overall], c1c7])

df_patient_pred = df_pred.groupby('StudyInstanceUID').apply(lambda df: patient_prediction(df)).to_frame('pred').join(df_pred.groupby('StudyInstanceUID')[target_cols].mean())

In [70]:
predictions = np.stack(df_patient_pred.pred.values.tolist())
predictions

array([[6.10107422e-01, 4.17480469e-02, 1.17187500e-01, 4.64477539e-02,
        3.47595215e-02, 7.01293945e-02, 1.56616211e-01, 3.61572266e-01],
       [6.99462891e-01, 4.61425781e-02, 1.93359375e-01, 5.61828613e-02,
        1.02722168e-01, 1.71264648e-01, 1.70776367e-01, 3.29101562e-01],
       [5.93505859e-01, 3.64990234e-02, 9.00268555e-02, 5.33447266e-02,
        1.15966797e-01, 1.33300781e-01, 1.27563477e-01, 2.67578125e-01],
       [6.13037109e-01, 1.40380859e-01, 3.34716797e-01, 3.99475098e-02,
        2.81982422e-02, 7.65991211e-02, 7.01293945e-02, 1.54296875e-01],
       [6.78955078e-01, 3.54919434e-02, 1.82373047e-01, 7.67211914e-02,
        1.06506348e-01, 1.37451172e-01, 1.02539062e-01, 3.62548828e-01],
       [4.19921875e-01, 1.27639771e-02, 1.20697021e-02, 6.38961792e-03,
        1.95617676e-02, 4.56542969e-02, 8.85009766e-02, 2.98095703e-01],
       [8.46069336e-01, 8.45947266e-02, 3.13232422e-01, 1.24450684e-01,
        2.02514648e-01, 2.58544922e-01, 2.74169922e-01, 3.

In [71]:
targets = df_patient_pred[target_cols].values
targets

array([[0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 1., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 1., 1., 0., 0., 0., 0., 0.],
       [1., 0., 1., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.]])

In [72]:
print('CV score:', weighted_loss(torch.logit(torch.as_tensor(predictions)).to(DEVICE), torch.as_tensor(targets).to(DEVICE)))

CV score: tensor(0.5256, device='cuda:0', dtype=torch.float64)
